# Relazione terzo progetto algoritmi avanzati

Francesca Meneghello 1227939 <br />
Leonardo Pratesi 1237582

## 1. Introduzione

Una iniziale implementazione prevedeva l'utilizzo di un dizionario contente coppie chiave valore ('id nodo' : \[array nodi adiacenti\]), in modo random veniva selezionata una chiave dall'array e successivamente un nodo adiacente dall'array di nodi adiacenti in modo da selezionare un arco randomico. Le chiavi dei due nodi venivano eliminate dal dizionario e veniva creata una nuova chiave fittizia che identificava il nodo nato dalla procedura di Contraction.
Successivamente tutti i valori delle chiavi venivano analizzati per sostiuire tutte le occorrenze dei due nodi eliminato con il nodo fittizio.
Tuttavia questa implementazione si è dimostrata molto inefficente dal punto di vista computazionale ed inoltre la selezione dell'arco casuale non rispettava la condizione di probabilità uniforme richiesta dall'algoritmo di Karger. <br />
Si è optato quindi per un implementazione attraverso una semplice lista che conserva tutti gli archi come coppie di tuple (nodo, nodo) che si è dimostrata molto più efficiente. Anche se gli archi non sono orientati per diminuire gli elementi della lista abbiamo tenuto una sola direzione, in quanto non cambiama il risultato dell'algoritmo. <br />
La scelta di memorizzare solo la lista di archi e non una lista di adiacenza ci ha permesso in fase di cancellazione e modifica degli archi di agire solo su un lista e non passare in rassegna i nodi e le lore liste di adiacenza. Oltre a ciò, ogni qual volta si creava un nuovo vertice per l'effetto della Contraction, si perdeva il suo riferimento dentro la lista di adiacenza per cui non era possibile accedervi in tempo costante ma occerreva passare tutta la lista.

In [1]:
import Contraction as c
import math
import time

lista, nodi= c.ListEdge('input_random_1_6.txt')
print(lista)

[('2', '5'), ('1', '4'), ('2', '4'), ('4', '5'), ('1', '3'), ('2', '6'), ('3', '5'), ('5', '6'), ('1', '2')]


# Full Contraction

L'algorimto di Full Contraction prende in input il nome del file contenente la matrice di adiacenza, genera la lista di archi attraverso il metodo ListEdge e salva il numero totale di nodi nel grafo. <br />
Successivamente l'algoritmo seleziona un arco casuale della lista e lo elimina insieme a tutto i suoi duplicati ed elimina i due nodi. <br />
Infine attraverso il metodo *changeEdge* modifica tutti gli archi adiacenti ai due nodi eliminati in modo che si riferiscano al nuovo nodo creato dalla Contraction. <br />
Il ciclo *while* viene eseguito finchè non rimangono solamente due vertici all'interno del grafo, tramite il contatore *nodes* infatti teniamo conto dei nodi presenti nel grafo. <br />
L'algoritmo restituisce il numero finale di archi presenti tra i due vertici rimasti.



In [2]:
def FullContraction(filename):
    edges, nodes= ListEdge(filename) #lista archi #
    while nodes>2:
        x= random.choice(edges) ## estrazione casuale arco ##
        nodea= x[0]
        nodeb= x[1]
        #### rimuovere arco selezionato e gli eventuali duplicati ####
        while x in edges:
            edges.remove(x)
        nodes=nodes-1
        ### sostituire con nuovo nodo e riposizionare gli archi corretti ###
        edges=changeEdge(edges, nodea, nodeb)  
        
    #print('Final result: ', len(edges))
    return len(edges)

# Karger
L'algoritmo di karger prende in input il nome del file contenente la matrice di adiacenza e il numero k che definisce quante volte eseguire la procedura di full contraction e ritorna il taglio minimo trovato ed il tempo impegato per trovare la migliore soluzione possibile, se questa non è stata trovata ritorna allora -1.
E' stato settato un timeout che interrompe l'algoritmo dopo 600 secondi.

In [3]:
import time
import Contraction as c
MAX= 9223372036854775807


def karger(filename, k):
    timeinit = time.time()
    #variable to save only the first time the correct solution is found
    found= False
    #the time of the first correct solution is inialized as -1 , so it is easy to check if the correct solution was not found
    timeright = -1
    #the method getrealresults simply gets the correct solution from the output file
    REAL = c.getrealresult(filename)
    min= MAX

    while k>0:
        #timeout to stop the algorithm
        if time.time() > timeinit + 600:
            break
        edges=c.FullContraction(filename)
        #condition to save the time of the first correct solution
        if REAL == edges and found == False:
            timeright = time.time()
            found = True
        if edges<min:
            min=edges
        k=k-1
    print('MINCUT: ' , min)
    return min, timeright

karger('input_random_6_10.txt', 115)



MINCUT:  3


(3, 1593768882.0175023)

# Numero Iterazioni
Il parametro k è stato selezionato per ogni grafo in base al numero di nodi in modo da assicurare una probabilità d' errore, nel calcolo del mincut, pari a *1/n* con *n*. 
Per questo motivo k viene impostato dal metodo defineK(n) che prende il numero 'n' di nodi del grafo e lo imposta a *((n x n)/2) x ln(n)* secondo la formula di probabilità affrontata nelle lezioni teoriche.  

In [4]:
def defineK(n):
    k= ((n*n)/2)* math.log(n)
    print('number of contractions: ', round(k))
    return round(k)


listfile = 'input_random_10_25.txt'
print(listfile)
start_time = time.time()
partialresult, solvetime = karger(listfile, defineK(c.nodfromname(listfile)))
end = time.time()
print("time : %s seconds " % (end - start_time))
if (solvetime==-1):
    print('NOT FOUND')
else:
    print("time first minimal solution: %s seconds " % (solvetime - start_time))

input_random_10_25.txt
number of contractions:  1006
MINCUT:  6
time : 1.0995464324951172 seconds 
time first minimal solution: 0.008315086364746094 seconds 


## 2. Il tempo impiegato dalla procedura di Full Contraction


Nel grafico seguente riportiamo la media del tempo impiegato da Full Contraction sui grafi della stessa dimensione. La curva cresce lentamente per i grafi fino a 100 vertici per poi aumentare rapidamente con i grafi più grandi.  </br>
I grafi con 200 vertici, infatti, impiegano poco meno di 2 secondi con conseguenze molto negative per l’algoritmo di Karger che ripete la FullContraction molte volte. <br />
La complessità risulta essere nel caso peggiore *0(n x m)* in quanto:

- il ciclo while viene ripetuto *(n-2)* volte
- la procedura ChangeEdge viene ripetuta per tutti gli archi non ancora eliminati, quindi all'inizio dell'algoritmo avremo *m* ripetizioni.


Tuttavia man mano che procediamo con la Full Contraction il numero totale di archi sarà sempre più piccolo, solo nel caso peggiore avremo *m* ripetizioni per la ChangeEdge. 

<img src="FullContractionMedia.png" width=550 height=450 />

## 3. Il tempo impiegato dall'algoritmo completo per ripetere la contrazione un numero sufficientemente alto di volte

Di seguito sono riportati diversi grafici che confrontano i tempi totali dell'algoritmo di Karger, i risultati non sono stati rappresentati tutti insieme perché presentavano scale molto diverse. Inoltre solo per i grafi con meno di cento nodi è stato possibile calcolare Karger entro il timeout prefissato, mentre per i rimanenti grafi sono state calcolate delle stime inseirte nella tabella seguente. <br />
Fino a 25 vertici l'algoritmo impiega meno di un secondo mentre per i grafi con 50 vertici impiega quasi un minuto, questo soprattutto a causa di *k* che passa da 1006 a 4890. 

<img src="PlotKarger.png" width=550 height=450 />

Nella tebella sono riportati le stime del tempo totale impiegato da Karger in minuti, calcolato con il tempo inpiagato da una singola Full Contraction per il numero totale di ripetizioni. Il tempo totale risulta molto elevato e oneroso già con 100 vertici tale da rendere l'algoritmo inutilizzabile. <br />
La complessità per l'algortimo di karger risulta essere *O(n x m x k)* e sostituendo *k* abbiamo *O(n^3 x m x log n)*. Tale complessità è data semplicemente dal costo della Full Contraction per il parametro *k*, il numero di volte che viene ripetuta per avere una probabilità di errore pari a *1/n*.

<img src="TabellaStime.png" width=450 height=350 />

<img src="PlotKarger2.png" width=450 height=350 />

## 4. Il discovery time, ossia il momento in cui l'algoritmo trova per la prima volta il taglio di costo mimimo

Come si può osservare dai dati ottenuti la soluzione ottima viene sempre identificata e il tempo impiegato mediamente è molto inferiore al tempo totale di esecuzione (nel caso dei grafi da 200 nodi dove l'algoritmo impiega 3000 minuti, la soluzione ottima viene trovata solo dopo 60 secondi. Data la grande discrepanza tra tempo totale e tempo impiegato per trovare la prima istanza della soluzione ottima non è stato possibile fare un confronto diretto ma ci siamo limitati a mostrare il tempo impiegato dalla soluzione ottima.


<img src="optsol1.png" width=450 height=350 />

<img src="optsol2.png" width=450 height=350 />

## 5. L'errore nella soluzione trovata rispetto al risultato ottimo

Nonostante l'analisi probabilistica richiedesse un numero elevato di iterazioni per ottenere la soluzione ottima con un'elevata confidenza, si osserva empiricamente che le soluzioni ottime vengono identificate in un numero molto minore di k-iterazioni. L'errore settando k= n\*n/2\ln(n) è pari a ZERO, l'algoritmo non sbaglia mai. Settando un k uniforme per tutti grafi i risultati sono comunque notevoli. Con k = 100 l'errore è ancora pari a ZERO, abbiamo quindi optato per un'analisi dell'errore per k=10 in modo da ottenere degli errori significativi.

<img src="errors.png" width=650 height=350 />

Come si osserva dalla statistica degli errori, anche con solo 10 iterazioni quasi tutti i grafi con numero di nodi minore di 100 vengono analizzati con successo dall'algoritmo di karger che riporta un errore pari a zero, per i grafi di dimensione maggiore invece l'errore segue un andamento imprevedibile ma rimane comunque sotto la soglia del 35%

## 6. Conclusioni

L'algoritmo di Karger dimostra come l'approccio randomico possa essere molto efficace nella risoluzione di determinati problemi, con i giusti parametri riesce ad assicurare un ampia precisione della soluzione ottima.